<a href="https://colab.research.google.com/github/petrtsv/aipoetry/blob/main/markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install markovify

In [2]:
!pip install russtress

In [3]:
import markovify
from markovify.chain import BEGIN, END

In [4]:
text = ""
with open('filtred_lines.txt', 'r') as f:
  text = f.read()

In [5]:
model = markovify.NewlineText(text, retain_original=1)

text = list(set(text.split()))

In [6]:
def get_choices(model, state):
        """
        Given a state, give the list of next words and weight of this words.
        """
        if model.chain.compiled:
          choices, _ = model.chain.model[state]
          weights = [1]*len(choices)
        elif state == tuple([ BEGIN ] * model.chain.state_size):
          choices = model.chain.begin_choices
          weights = [1]*len(choices)
        else:
          choices, weights = zip(*model.chain.model[state].items())
        choices = list(choices)
        weights = list(weights)
        lst = list(zip(weights, choices))
        lst.sort()
        choices = [ch for _, ch in lst]
        return choices

In [7]:
from tqdm import tqdm

from russtress import Accent
accent = Accent()

def is_rhyme(word_a, word_b):
  # оставляем только суффикс сразу после ударения
  word_a = word_a[accent.put_stress(word_a).find('`'):]
  word_b = word_b[accent.put_stress(word_b).find('`'):]
  return word_a == word_b

rhyme = dict()

text = get_choices(model, (BEGIN, '$'))

In [8]:
import random

def generate_rhyme_line(model, prev_line):
  prev_word = prev_line.split()[0]
  choices = get_choices(model, (BEGIN, '$'))[:100]
  rhyme_choices = [word for word in choices if is_rhyme(word, prev_word)]
  if rhyme_choices == []:
    rhyme_choices = choices
  rhyme_word = rhyme_choices[random.randint(0, len(rhyme_choices)-1)]
  result = [rhyme_word]
  length = random.randint(4, 6)
  state = ('$', rhyme_word)
  for i in range(length):
    try:
      result += model.chain.walk((state))
    except:
      break
    state = (state[1], result[-1])
  result = ' '.join(result)
  return result

In [ ]:
for i in range(20):
  first_line = model.make_sentence()
  first_line = ' '.join(first_line.split()[1:])
  second_line = generate_rhyme_line(model, first_line)
  first_line = ' '.join(first_line.split()[::-1])
  second_line = ' '.join(second_line.split()[::-1])
  print(first_line)
  print(second_line)


возьмет ли арфу дивной силой укроти
в час торжеств адонаи
и мы получим этот край назвал бы зурбаганом
штыки горят под солнцем азиатским
и чешется у вас еще достаточно делов
и ста вестминстерских аббатств
однакож как ни страшна
и в ответ ага
и мы дрожим как маленькие черти
не надо долгой агитации
почтенный человек помог ему заступник честно слово
можно б административно
не в минуте живем а мы так не похоже на соседний пырей
таруса где подписью в и прусский и прусский и австрийский
мужицким пахнет варевом костры в дыму угроз нескромных
в мечтах абстрактных
даже писем мне было мало или много
без претензий на авторство
ты друг мой спит
когда он был человек тот авантюрист
я буду таким как ты изменился
да колен и как стройна гляди аглая
но все ж хочу
обращено к александру абрамовичу
но дерево растет напротив
и ста вестминстерских аббатств
гудки звонки и голоса людей
а мальчик то же говорят абстрактный
